# Chat dialog

Guidance supports chat-based models like ChatGPT and GPT-4 using role tags. These are then converted to the appropriate format for the model (either a JSON API format or special tokens).

In [ ]:
import logging

In [1]:
call_delay_secs = 0
requested_log_level = logging.WARNING

In [ ]:
logging.basicConfig(level=requested_log_level)

In [2]:
import os

# Uncomment if using DefaultAzureCredential below
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# This is the name of the model deployed, such as 'gpt-4' or 'gpt-3.5-turbo
model = os.getenv("AZUREAI_CHAT_MODEL", "Please set the model")

# This is the deployment URL, as provided in the Azure AI playground ('view code')
# It will end with 'openai.azure.com'
azure_endpoint = os.getenv("AZUREAI_CHAT_BASE_ENDPOINT", "Please set the endpoint")

# This is the name of the deployment specified in the Azure portal
azure_deployment = os.getenv("AZUREAI_CHAT_DEPLOYMENT", "Please set the deployment name")

# This is the deployed API version, such as 2024-02-15-preview
azure_api_version = os.getenv("AZUREAI_CHAT_API_VERSION", "Please set the API version")

# The environment variable should be set to the API key from the Azure AI playground:
# api_key=os.getenv("AZUREAI_CHAT_KEY", "Please set API key")

# Alternatively, we can use Entra authentication
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default"
)

Preliminaries concluded, we can now create our model:

In [3]:
from guidance import models, gen

azureai_model = models.AzureOpenAI(
    model=model,
    azure_endpoint=azure_endpoint,
    azure_deployment=azure_deployment,
    version=azure_api_version,
    # For authentication, use either
    azure_ad_token_provider=token_provider,
    # or
    # api_key=api_key,
)

# Multi-step chat with hidden blocks

We are now going to set up a multistage chat, where we have the chat bot help the use achieve some goal.
The user will only have to specify the goal, and then we will create a chain-of-thought conversation with the bot which will:

1. Ask the bot for a number of suggestions.
2. List the pros and cons of each.
3. Pick the best suggestion.
4. Product a detailed action plan.

Our goal is to only show the final result to the user.

Now, let us define our generation function:

In [4]:
import re
import time

import guidance
from guidance import gen, select, system, user, assistant

@guidance
def plan_for_goal(lm, goal: str):
    
    # This is a helper function which we will use below
    def parse_best(prosandcons, options):
        best = re.search(r'Best=(\d+)', prosandcons)
        if not best:
            best =  re.search(r'Best.*?(\d+)', 'Best= option is 3')
        if best:
            best = int(best.group(1))
        else:
            best = 0
        return options[best]

    # Some general instruction to the model
    with system():
        lm += "You are a helpful assistant."

    # Simulate a simple request from the user
    # Note that we switch to using 'lm2' here, because these are intermediate steps (so we don't want to overwrite the current lm object)
    with user():
        lm2 = lm + f"""\
        I want to {goal}
        Can you please generate one option for how to accomplish this?
        Please make the option very short, at most one line."""

    # Generate several options. Note that this means several sequential generation requests
    n_options = 5
    with assistant():
        options = []
        for i in range(n_options):
            options.append((lm2 + gen(name='option', temperature=1.0, max_tokens=50))["option"])

    # Have the user request pros and cons
    with user():
        lm2 += f"""\
        I want to {goal}
        Can you please comment on the pros and cons of each of the following options, and then pick the best option?
        ---
        """
        for i, opt in enumerate(options):
            lm2 += f"Option {i}: {opt}\n"
        lm2 += f"""\
        ---
        Please discuss each option very briefly (one line for pros, one for cons), and end by saying Best=X, where X is the number of the best option."""

    # Get the pros and cons from the model
    with assistant():
        lm2 += gen(name='prosandcons', temperature=0.0, max_tokens=600, stop="Best=") + "Best=" + gen("best", regex="[0-9]+")
        time.sleep(call_delay_secs)

    # The user now extracts the one selected as the best, and asks for a full plan
    # We switch back to 'lm' because this is the final result we want
    with user():
        lm += f"""\
        I want to {goal}
        Here is my plan: {options[int(lm2["best"])]}
        Please elaborate on this plan, and tell me how to best accomplish it."""

    # The plan is generated
    with assistant():
        lm += gen(name='plan', max_tokens=500)
        time.sleep(call_delay_secs)

    return lm

Create a plan for the user. Note how the portions which were sent to `lm2` in the function above are not shown in the final result:

In [5]:
results = azureai_model + plan_for_goal(goal="read more books")

We can access the final plan itself:

In [6]:
print(results['plan'])

Absolutely, setting a daily reading goal is a great way to incorporate more reading into your routine. Here's a more detailed plan:

1. **Choose the Right Book**: Start with a book that interests you. It could be a novel, a self-help book, a biography, or any genre that you enjoy. The key is to choose a book that you're excited to read, as this will make it easier to stick to your reading plan.

2. **Set a Specific Goal**: You've mentioned reading one chapter or for 30 minutes each day. This is a great start. Make sure your goal is achievable and not too overwhelming. If a chapter is too long, you can set a goal to read a certain number of pages each day.

3. **Create a Reading Schedule**: Decide on a specific time to read each day. It could be in the morning, during lunch breaks, or before bed. Consistency is key, so try to stick to this schedule as closely as possible.

4. **Find a Quiet Place**: Choose a quiet and comfortable place to read where you won't be easily distracted. This 

## Asking help from experts

Now, let us ask our chat model to pick some experts in a particular field, and impersonate them to give advice:

In [7]:
@guidance
def run_expert_advice(lm, query: str):
    # Some general instruction to the model
    with system():
        lm += "You are a helpful assistant."

    with user():
        lm += f"""I want a response to the following question:
{query}
Who are 3 world-class experts (past or present) who would be great at answering this?
Please don't answer the question or comment on it yet.
"""

    with assistant():
        lm += gen(name='experts', temperature=0, max_tokens=300)
        time.sleep(call_delay_secs)

    with user():
        lm += """Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
Please start your answer with ANSWER:
"""

    with assistant():
        lm += gen(name='answer', temperature=0, max_tokens=500)
        time.sleep(call_delay_secs)

    return lm

In [8]:
mean_life = azureai_model + run_expert_advice("What is the meaning of life?")

In [9]:
more_productive = azureai_model + run_expert_advice('How can I be more productive?')

## Agents

We are now going to define a 'conversation agent.'
This maintains a memory of a conversation, and can generate an appropriate reply, based on the persona it has been given.

In [10]:
class ConversationAgent:
    def __init__(self, chat_model, name: str, instructions: str, context_turns: int = 2):
        self._chat_model = chat_model
        self._name = name
        self._instructions = instructions
        self._my_turns = []
        self._interlocutor_turns = []
        self._went_first = False
        self._context_turns = context_turns

    @property
    def name(self) -> str:
        return self._name
    
    def reply(self, interlocutor_reply = None) -> str:
        if interlocutor_reply is None:
            self._my_turns = []
            self._interlocutor_turns = []
            self._went_first = True
        else:
            self._interlocutor_turns.append(interlocutor_reply)

        # Get trimmed history
        my_hist = self._my_turns[(1-self._context_turns):]
        interlocutor_hist = self._interlocutor_turns[-self._context_turns:]

        # Set up the system prompt
        curr_model = self._chat_model
        with system():
            curr_model += f"Your name is {self.name}. {self._instructions}"
            if len(interlocutor_hist) == 0:
                curr_model += "Introduce yourself and start the conversation"
            elif len(interlocutor_hist) == 1:
                curr_model += "Introduce yourself before continuing the conversation"

        # Replay the last few turns
        for i in range(len(my_hist)):
            with user():
                curr_model += interlocutor_hist[i]
            with assistant():
                curr_model += my_hist[i]

        if len(interlocutor_hist) > 0:
            with user():
                curr_model += interlocutor_hist[-1]

        with assistant():
            curr_model += gen(name='response', max_tokens=100)
        time.sleep(call_delay_secs)

        self._my_turns.append(curr_model['response'])
        return curr_model['response']

We can have two of these agents converse with each other with a _conversation simulator_:

In [11]:
def conversation_simulator(
    bot0: ConversationAgent,
    bot1: ConversationAgent,
    total_turns: int = 5 ):
    conversation_turns = []
    last_reply = None
    for _ in range(total_turns):
        last_reply = bot0.reply(last_reply)
        conversation_turns.append(dict(name=bot0.name, text=last_reply))
        time.sleep(call_delay_secs)
        last_reply = bot1.reply(last_reply)
        conversation_turns.append(dict(name=bot1.name, text=last_reply))
    return conversation_turns

Now, let's try generating a conversation:

In [12]:
bot_instructions = """You are taking part in a discussion about bodyline bowling.
Only generate text as yourself and do not prefix your reply with your name.
Keep your answers to a couple of short sentences."""

bradman_bot = ConversationAgent(azureai_model, "Donald Bradman", bot_instructions, context_turns=5)
jardine_bot = ConversationAgent(azureai_model, "Douglas Jardine", bot_instructions, context_turns=5)

conversation_turns = conversation_simulator(bradman_bot, jardine_bot, total_turns=3)

In [13]:
for turn in conversation_turns:
    print(f"{turn['name']}: {turn['text']}\n")

Donald Bradman: Hello everyone, I'm Donald Bradman, a former Australian cricketer. I'd like to discuss the controversial topic of bodyline bowling. What are your thoughts on this strategy?

Douglas Jardine: Hello Donald, I'm Douglas Jardine, a former English cricketer. I believe bodyline bowling was a legitimate strategy, albeit a controversial one. It was a tactical response to the dominant batting during that era.

Donald Bradman: Hello Douglas, I respect your viewpoint. However, I believe that while it may have been a tactical response, bodyline bowling was a dangerous strategy that risked the safety of the batsmen.

Douglas Jardine: I understand your concerns, Donald. The safety of players is indeed paramount. However, it's important to remember that bodyline bowling was within the rules of the game at that time. It was a tactical innovation, albeit a contentious one.

Donald Bradman: Indeed, Douglas, it was within the rules. However, the spirit of the game should always be upheld,

<hr style="height: 1px; opacity: 0.5; border: none; background: #cccccc;">
<div style="text-align: center; opacity: 0.5">Have an idea for more helpful examples? Pull requests that add to this documentation notebook are encouraged!</div>